In [1]:
import sys
import logging

import datasets
from datasets import load_dataset
import torch
import transformers
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import os
import json
import wandb


In [ ]:
#!pip install -U torch

In [ ]:
# !pip install datasets transformers trl wandb

In [2]:
ds = load_dataset("NousResearch/hermes-function-calling-v1", "glaive_func_calling")

In [3]:
from ast import literal_eval
tools_list = []
for tool in ds['train']['tools']:
    if tool == 'null':
        tools_list.append(None)
        continue

    # str fixes to convert to dict
    tool = tool.replace('true','True')
    tool = tool.replace('false','False')
    tool = tool.replace('null','None')
    tools_list.append(literal_eval(tool))   
        


In [4]:
tools_list[123]

[{'type': 'function',
  'function': {'name': 'get_stock_price',
   'description': 'Get the current stock price',
   'parameters': {'type': 'object',
    'properties': {'symbol': {'type': 'string',
      'description': 'The stock symbol, e.g. AAPL'}},
    'required': ['symbol']}}},
 {'type': 'function',
  'function': {'name': 'search_recipes',
   'description': 'Search for recipes based on ingredients',
   'parameters': {'type': 'object',
    'properties': {'ingredients': {'type': 'array',
      'items': {'type': 'string'},
      'description': 'The ingredients to be used in the recipes'}},
    'required': ['ingredients']}}}]

In [5]:
pydantic_schema = """{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}"""
from tqdm import tqdm

system_prompts = []
for i in tqdm(range(len(ds['train']))):
    if ds['train']['tools'][i] == 'null':
        system_prompts.append(system_prompt)
    else:
        available_tools = tools_list[i]
        system_prompt = f"""You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
{available_tools}
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{pydantic_schema}
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{{tool_call}}
</tool_call>
"""
        system_prompts.append(system_prompt)

100%|██████████| 5209/5209 [00:20<00:00, 258.50it/s]


In [6]:
#  To use as reference when creating tools in the future.

custom_tools_list = [{'type': 'function',
  'function': {'name': 'turn_on_lights',
   'description': 'Turn the lights on',
    'parameters': {}}},    
 {'type': 'function',
  'function': {'name': 'get_news',
   'description': 'Get the latest news',
   'parameters': {'type': 'object',
    'properties': {'category': {'type': 'string',
      'description': 'The category of news, e.g. sports, politics'}},
    'required': ['category']}}}]

In [7]:
# add column to dataset called system_prompt
ds['train'] = ds['train'].add_column('system_prompt', system_prompts)


In [43]:

###################
# Tokenizer Loading
###################

#checkpoint_path = "HuggingFaceTB/SmolLM-1.7B-instruct"
checkpoint_path = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 2048
tokenizer.pad_token = "<unk>"  # note this is specific to smollm
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token )
tokenizer.padding_side = 'right'
# https://stackoverflow.com/questions/76446228/setting-padding-token-as-eos-token-when-using-datacollatorforlanguagemodeling-fr


In [44]:
tokenizer.eos_token_id, tokenizer.pad_token_id

(32000, 0)

In [45]:
tokenizer

LlamaTokenizerFast(name_or_path='microsoft/Phi-3.5-mini-instruct', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	

In [46]:
##################
# Data Processing
##################
def clean_mesages(messages, updated_system_prompt):
    messages_clean = []
    for m in messages:
        role = m['from']
        content = m['value']
        if role == 'human':
            role = 'user'
        elif role == 'gpt':
            role = 'assistant'
        elif role == 'system':
            content = updated_system_prompt
        messages_clean.append({'role': role, 'content': content})
    return messages_clean

def apply_chat_template(
    example,
    tokenizer,
):
    updated_system_prompt = example["system_prompt"]
    messages_init = example["conversations"]
    messages = clean_mesages(messages_init, updated_system_prompt)
    example["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False).strip('\n')
    return example



train_dataset = ds['train']
column_names = list(train_dataset.features)

processed_train_dataset = train_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    desc="Applying chat template to train_sft",
)

# shuffle the dataset
processed_train_dataset = processed_train_dataset.shuffle(seed=42)

In [47]:
processed_train_dataset[0]

{'id': '3faaadfc-6ccc-47ee-9b54-77b5a333a73b',
 'conversations': [{'from': 'system',
   'value': 'You are a helpful assistant, with no access to external functions.'},
  {'from': 'human',
   'value': 'Transform these sentences into a passive voice\nThey are singing a song.'},
  {'from': 'gpt', 'value': 'A song is being sung by them.'},
  {'from': 'human', 'value': 'What is passive voice and why is it used?'},
  {'from': 'gpt',
   'value': 'In passive voice, the subject of a sentence receives the action instead of performing it. Passive voice is used to shift focus from the doer (subject) of an action to the receiver (object). It can also be used to make writing more formal or objective.'},
  {'from': 'human',
   'value': 'How can I identify passive voice in a sentence?'},
  {'from': 'gpt',
   'value': 'Passive voice can be identified by looking for a form of "to be" (is, was, are, were, be, being, been) followed by a past participle verb (usually ending in -ed or -en) and the absence o

In [49]:
print(processed_train_dataset['text'][13])

<|system|>
You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
[{'type': 'function', 'function': {'name': 'search_wikipedia', 'description': 'Search for a topic on Wikipedia', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query'}}, 'required': ['query']}}}, {'type': 'function', 'function': {'name': 'search_recipes', 'description': 'Search for recipes based on ingredients and dietary restrictions', 'parameters': {'type': 'object', 'properties': {'ingredients': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The ingredients available'}, 'dietary_restrictions': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The dietary restrictions to consider'}}, 'required': ['ingredients', 'd

# Perfect assistant example:

<|im_start|>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'create_todo', 'description': 'Create a new todo item', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the todo item'}, 'description': {'type': 'string', 'description': 'The description of the todo item'}}, 'required': ['title']}}}, {'type': 'function', 'function': {'name': 'create_invoice', 'description': 'Create a new invoice', 'parameters': {'type': 'object', 'properties': {'client': {'type': 'string', 'description': 'The name of the client'}, 'items': {'type': 'array', 'items': {'type': 'object', 'properties': {'description': {'type': 'string', 'description': 'The description of the item'}, 'quantity': {'type': 'integer', 'description': 'The quantity of the item'}, 'price': {'type': 'number', 'description': 'The price of the item'}}, 'required': ['description', 'quantity', 'price']}, 'description': 'The list of items in the invoice'}}, 'required': ['client', 'items']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call><|im_end|>
<|im_start|>user
I need to create a new todo item.<|im_end|>
<|im_start|>assistant
Sure, I can help with that. Could you please provide me with the title and description of the todo item?<|im_end|>
<|im_start|>user
The title is "Grocery Shopping" and the description is "Buy fruits, vegetables, and bread".<|im_end|>
<|im_start|>assistant
<tool_call>
{'name': 'create_todo', 'arguments': {'title': 'Grocery Shopping', 'description': 'Buy fruits, vegetables, and bread'}}
</tool_call><|im_end|>
<|im_start|>tool
<tool_response>
{'status': 'success', 'message': "Todo item 'Grocery Shopping' has been created successfully."}
</tool_response><|im_end|>
<|im_start|>assistant
Your todo item "Grocery Shopping" has been created successfully. It includes "Buy fruits, vegetables, and bread".<|im_end|>


This todo list creation is exactly what you'd want an assistant to help with

In [18]:
# Now test how this works with the existing model

####################
# Base Model Loading
####################
checkpoint_path = "HuggingFaceTB/SmolLM-1.7B"
#checkpoint_path="microsoft/Phi-3.5-mini"
#checkpoint_path = "microsoft/Phi-3.5-mini-instruct"

model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
#    attn_implementation="flash_attention_2",  # only works on latest gpus, probably not worth it in most cases
     torch_dtype=torch.bfloat16,
   device_map='auto'
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
user_prompt = "Did the nets win?"

#  To use as reference when creating tools in the future.

available_tools = [{'type': 'function',
  'function': {'name': 'turn_on_lights',
   'description': 'Turn the lights on',
    'parameters': {}}},    
 {'type': 'function',
  'function': {'name': 'get_news',
   'description': 'Get the latest news',
   'parameters': {'type': 'object',
    'properties': {'category': {'type': 'string',
      'description': 'The category of news, e.g. sports, politics'}},
    'required': ['category']}}}]


system_prompt = f"""You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
{available_tools}
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{pydantic_schema}
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{{tool_call}}
</tool_call>
"""

tool_calling_system_prompt = f"""<|im_start|>system
{system_prompt}<|im_end|>"""
tool_calling_user_prompt = f"""
<|im_start|>user
{user_prompt}<|im_end|>
<|im_start|>assistant
"""

tool_calling_prompt = tool_calling_system_prompt + tool_calling_user_prompt


model.eval();

input_ids = tokenizer.encode(tool_calling_prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
print(output_text)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(formatted_output_text)

<|im_start|>system
You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
[{'type': 'function', 'function': {'name': 'turn_on_lights', 'description': 'Turn the lights on', 'parameters': {}}}, {'type': 'function', 'function': {'name': 'get_news', 'description': 'Get the latest news', 'parameters': {'type': 'object', 'properties': {'category': {'type': 'string', 'description': 'The category of news, e.g. sports, politics'}}, 'required': ['category']}}}]
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}
For each function call return a js

In [ ]:
logger = logging.getLogger(__name__)
wandb.init(project="smollm-ft-function-calling")
###################
# Hyper-parameters
###################
training_config = {
    "do_eval": False,
    "learning_rate": 5.0e-04,
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 2,
    "log_level": "info",
    "logging_steps": 100,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 5,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "remove_unused_columns": True,
    "save_steps": 500,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.05,
    "report_to":"wandb"
    }


train_conf = TrainingArguments(**training_config)

###############
# Setup logging
###############
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = train_conf.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

logger.info(f"Training/evaluation parameters {train_conf}")


In [ ]:
###########
# Training
###########

model.train();
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    train_dataset=processed_train_dataset,
    max_seq_length=2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    #packing=True,
)
train_result = trainer.train()
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()


In [21]:
# Load the model from the checkpoint

# find most recently created folder in checkpoint_dir and set as checkpoint path
#checkpoint_path = sorted(os.listdir(train_conf.output_dir))[-1]
checkpoint_path = os.path.join('checkpoint_dir', 'checkpoint-4500')
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
     torch_dtype=torch.bfloat16,
   device_map='auto'
)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, **model_kwargs)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
tokenizer

LlamaTokenizerFast(name_or_path='checkpoint_dir/checkpoint-4500', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	3

In [ ]:
user_prompt = "Did the nets win?"

#  To use as reference when creating tools in the future.

available_tools = [{'type': 'function',
  'function': {'name': 'turn_on_lights',
   'description': 'Turn the lights on',
    'parameters': {}}},    
 {'type': 'function',
  'function': {'name': 'get_news',
   'description': 'Get the latest news',
   'parameters': {'type': 'object',
    'properties': {'category': {'type': 'string',
      'description': 'The category of news, e.g. sports, politics'}},
    'required': ['category']}}}]


system_prompt = f"""You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
{available_tools}
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{pydantic_schema}
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{{tool_call}}
</tool_call>
"""

tool_calling_system_prompt = f"""<|im_start|>system
{system_prompt}<|im_end|>"""
tool_calling_user_prompt = f"""
<|im_start|>user
{user_prompt}<|im_end|>
<|im_start|>assistant
"""

tool_calling_prompt = tool_calling_system_prompt + tool_calling_user_prompt


model.eval();

input_ids = tokenizer.encode(tool_calling_prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
#print(output_text)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(formatted_output_text)

In [40]:
user_prompt = "Send a message to John saying hi"

#  To use as reference when creating tools in the future.

available_tools = [{'type': 'function',
  'function': {'name': 'send_message',
   'description': 'Send a message to a contact',
    'parameters': {'type': 'object',
    'properties': {'contact': {'type': 'string',
      'description': 'The contact to send the message to'},
     'message': {'type': 'string',
      'description': 'The message to send'}} ,
    'required': ['contact', 'message']}}},

 {'type': 'function',
  'function': {'name': 'get_news',
   'description': 'Get the latest news',
   'parameters': {'type': 'object',
    'properties': {'category': {'type': 'string',
      'description': 'The category of news, e.g. sports, politics'}},
    'required': ['category']}}}]


system_prompt = f"""You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
{available_tools}
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{pydantic_schema}
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{{tool_call}}
</tool_call>
"""

tool_calling_system_prompt = f"""<|im_start|>system
{system_prompt}<|im_end|>"""
tool_calling_user_prompt = f"""
<|im_start|>user
{user_prompt}<|im_end|>
<|im_start|>assistant
"""

tool_calling_prompt = tool_calling_system_prompt + tool_calling_user_prompt

tool_calling_prompt = tool_calling_prompt.replace("<|im_start|>system", "<|system|>").replace("<|im_start|>user", "<|user|>").replace("<|im_start|>assistant", "<|assistant|>").replace("<|im_end|>", "<|end|>")
model.eval();

input_ids = tokenizer.encode(tool_calling_prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
output = model.generate(input_ids, max_new_tokens=256,  do_sample=False, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False, pad_token_id = tokenizer.eos_token_id)
#print(output_text)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(output_text)

<|system|> You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
[{'type': 'function', 'function': {'name': 'send_message', 'description': 'Send a message to a contact', 'parameters': {'type': 'object', 'properties': {'contact': {'type': 'string', 'description': 'The contact to send the message to'}, 'message': {'type': 'string', 'description': 'The message to send'}}, 'required': ['contact', 'message']}}}, {'type': 'function', 'function': {'name': 'get_news', 'description': 'Get the latest news', 'parameters': {'type': 'object', 'properties': {'category': {'type': 'string', 'description': 'The category of news, e.g. sports, politics'}}, 'required': ['category']}}}]
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{'title':

In [36]:
print(tool_calling_prompt)

<|system|>
You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
[{'type': 'function', 'function': {'name': 'turn_on_lights', 'description': 'Turn the lights on in a room', 'parameters': {}}}, {'type': 'function', 'function': {'name': 'get_news', 'description': 'Get the latest news', 'parameters': {'type': 'object', 'properties': {'category': {'type': 'string', 'description': 'The category of news, e.g. sports, politics'}}, 'required': ['category']}}}]
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}
For each function call return a 

In [57]:
test_prompt = """<|system|>
You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
[{'type': 'function', 'function': {'name': 'search_wikipedia', 'description': 'Search for a topic on Wikipedia', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query'}}, 'required': ['query']}}}, {'type': 'function', 'function': {'name': 'search_recipes', 'description': 'Search for recipes based on ingredients and dietary restrictions', 'parameters': {'type': 'object', 'properties': {'ingredients': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The ingredients available'}, 'dietary_restrictions': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The dietary restrictions to consider'}}, 'required': ['ingredients', 'dietary_restrictions']}}}]
</tools>
Use the following pydantic model json schema for each tool call you will make: 
{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call>
<|end|>
<|user|>
Search recipes?<|end|>
<|assistant|>"""

input_ids = tokenizer.encode(test_prompt, return_tensors='pt')
input_ids = input_ids.to(model.device)
model.eval();

output = model.generate(input_ids, max_new_tokens=256,  do_sample=False)#, pad_token_id=tokenizer.eos_token_id)
output_text = tokenizer.decode(output[0], skip_special_tokens=False)#, pad_token_id = tokenizer.eos_token_id)
#print(output_text)
formatted_output_text = "<|im_end|>".join(output_text.split("<|im_end|>")[:3]) + "<|im_end|>"
print(output_text)

<|system|> You are a function calling AI model.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools:
<tools>
[{'type': 'function', 'function': {'name': 'search_wikipedia', 'description': 'Search for a topic on Wikipedia', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The search query'}}, 'required': ['query']}}}, {'type': 'function', 'function': {'name': 'search_recipes', 'description': 'Search for recipes based on ingredients and dietary restrictions', 'parameters': {'type': 'object', 'properties': {'ingredients': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The ingredients available'}, 'dietary_restrictions': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The dietary restrictions to consider'}}, 'required': ['ingredients', 'd

In [54]:
output

tensor([[32006,   887,   526,   263,   740,  5432,   319, 29902,  1904, 29889,
            13,  3492,   526,  4944,   411,   740,  1804,  3698,  2629,   529,
          8504,  2565,  8504, 29958,  6560,  8282, 29889,    13,  3492,  1122,
          1246,   697,   470,   901,  3168,   304,  6985,   411,   278,  1404,
          2346, 29889,  3872, 29915, 29873,  1207, 20813,  1048,   825,  1819,
           304, 18665,   964,  3168, 29889,  2266,   526,   278,  3625,  8492,
         29901,    13, 29966,  8504, 29958,    13, 29961, 10998,  1853,  2396,
           525,  2220,   742,   525,  2220,  2396, 11117,   978,  2396,   525,
          4478, 29918,  6011,   742,   525,  8216,  2396,   525,  7974,   363,
           263, 11261,   373, 14109,   742,   525, 16744,  2396, 11117,  1853,
          2396,   525,  3318,   742,   525, 11330,  2396, 11117,  1972,  2396,
         11117,  1853,  2396,   525,  1807,   742,   525,  8216,  2396,   525,
          1576,  2740,  2346, 29915, 11656,   525, 1